In [66]:
# Importamos todo lo que vamos a utilizar
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import WordNetLemmatizer
import emoji
import string
import random

En este módulo, realizaremos una función para que genere el resumen extractivo de un post de entrada basado en frecuencias. Para ello, tomaremos el post limpio como una cadena de texto y creamos un vector de conteo de su vocabulario mediante la aplicación de la técnica de Bag of Words. Una vez tengamos esto, para facilitar el análisis creamos un diccionario con el vocabulario y su frecuencia en la cadena de texto.
La idea principal de este proceso es que las oraciones del resumen sean aquellas que tengan mayor valor. Este valor viene dado por la importancia de sus palabras, es decir, por la cantidad de veces que aparece sus palabras en todo el post. Por ello, para cada oración realizamos un contador al que iremos sumando la distribución de frecuencia de cada palabra de la oración. 

In [67]:
# Cargar el dataset
dataframe = pd.read_csv('reddit_database_sentiment.csv', delimiter=';', low_memory=False)
dataframe.head()

,created_date,created_timestamp,subreddit,title,author,author_created_utc,full_link,score,num_comments,num_crossposts,subreddit_subscribers,post,sentiment
0,2010-02-11 19:47:22,1265910442.0,analytics,So what do you guys all do related to analytic...,xtom,1.227476e+09,https://www.reddit.com/r/analytics/comments/b0...,7.0,4.0,0.0,NaN,There's a lot of reasons to want to know all t...,NEGATIVE
1,2010-03-04 20:17:26,1267726646.0,analytics,"Google's Invasive, non-Anonymized Ad Targeting...",xtom,1.227476e+09,https://www.reddit.com/r/analytics/comments/b9...,2.0,1.0,0.0,NaN,"I'm cross posting this from /r/cyberlaw, hopef...",NEGATIVE
2,2011-01-06 04:51:18,1294282278.0,analytics,"DotCed - Functional Web Analytics - Tagging, R...",dotced,1.294282e+09,https://www.reddit.com/r/analytics/comments/ew...,1.0,1.0,NaN,NaN,"DotCed,a Functional Analytics Consultant, offe...",NEGATIVE
3,2011-01-19 11:45:30,1295430330.0,analytics,Program Details - Data Analytics Course,iqrconsulting,1.288245e+09,https://www.reddit.com/r/analytics/comments/f5...,0.0,0.0,NaN,NaN,Here is the program details of the data analyt...,NEGATIVE
4,2011-01-19 21:52:28,1295466748.0,analytics,potential job in web analytics... need to anal...,therewontberiots,1.278672e+09,https://www.reddit.com/r/analytics/comments/f5...,2.0,4.0,NaN,NaN,i decided grad school (physics) was not for me...,POSITIVE


Para la construcción de ese diccionario, vamos a comenzar limpiando un poco el texto con una función más simple que la que hemos hecho en el módulo 1. Una vez tengamos esto, aplicamos el vectorizador, extramos el vocabulario y su frecuencia y construimos el diccionario.

Tras esto, tokenizamos por oraciones e iremos sumando el valor de cada palabra de la oración. Formaremos un diccionario oracion-suma total. Finalmente miraremos las oraciones con mayor puntuación. Esas serán las que compongan nuestro resumen.

In [68]:
def post_summarisation(text:str):
    """Construcción del diccionario palabras-frecuencia"""
    # Limpieza del texto
    text_limpio = re.sub(r'\[[0-9]*\]', ' ', text)
    text_limpio = re.sub(r'\s+', ' ', text_limpio)
    text_limpio = text_limpio.lower()
    text_limpio = re.sub(r'\d', ' ', text_limpio)
    text_limpio = re.sub(r'\s+', ' ', text_limpio)
    text_limpio = re.sub(r'[^\w\s]', '', text_limpio)
    
    # Creación del diccionario (no tendremos en cuenta las stopwords)
    vectorizador = CountVectorizer(stop_words=stopwords.words('english'))
    frecuencias = vectorizador.fit_transform([text_limpio]) 
    vocabulario = list(vectorizador.vocabulary_.keys()) # Obtenemos el vocabulario
    diccionario_palabras = dict(zip(vocabulario, frecuencias.sum(axis=0).A1))

    """Construcción del diccionario con el valor de cada oración"""
    oraciones = sent_tokenize(text) # Segmentar el texto en oraciones
    diccionario_oraciones = {} # Creación del diccionario vacío
    for oracion in oraciones:
        for palabra, freq in diccionario_palabras.items():
            if palabra in oracion.lower():
                if oracion in diccionario_oraciones.keys():
                    diccionario_oraciones[oracion] += freq
                else:
                    diccionario_oraciones[oracion] = freq
    
    """Obtenemos un cierto número de esas oraciones una vez ordenadas según su valor.
       El número de oraciones será aleatorio entre 1 y el número de oraciones total."""
    oraciones_ordenadas = sorted(diccionario_oraciones, key=diccionario_oraciones.get, reverse=True)
    oraciones_resumen = oraciones_ordenadas[:random.randint(1, len(oraciones))]
    resumen = ' '.join(oraciones_resumen) # Devolvemos en forma de texto

    """Evaluación de la similitud entre el texto y el resumen generado"""
    tfidf = TfidfVectorizer()
    matriz = tfidf.fit_transform([text, resumen])
    similitud = cosine_similarity(matriz)
    return resumen, similitud

In [71]:
print('Texto: ', dataframe.post[32])
print()
print('Resumen:', post_summarisation(dataframe.post[32])[0])
print('Similitud:', post_summarisation(dataframe.post[32])[1])

Texto:  Greetings,

I'm trying to configure GA for a business that uses "yahoo small business" hosting and store.  They use their own domain for the store, but when someone wants to check out, it moves domains to yahoo's servers.  Is it possible to track the cart in the same GA account? Right now we have no goals working. 

Do I need to make a separate account for yahoo's cart domain? It also changes from unsecured to secured during checkout, I'm not sure if this will affect the setup either.

Thanks for any help guys!

TLDR - http://www.store.com &gt; https://www.store.yahoo.net Wat do?

Resumen: Greetings,

I'm trying to configure GA for a business that uses "yahoo small business" hosting and store.
Similitud: [[1.         0.97793952]
 [0.97793952 1.        ]]
